### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}" 
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)


In [4]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(0, 20):
    # create query with a page number
    query_url_pages = f"{query_url}&page={str(page)}"
    
    # API results show 10 articles at a time
    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(query_url_pages).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    # loop through the reviews["response"]["docs"] and append each review to the list
    # Print the page that was just retrieved
    try:
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
        print(f"checked page {page}")    
    # loop through the reviews["response"]["docs"] and append each review to the list
    except:
        print(f"result not found on page {page}")



checked page 0
checked page 1
checked page 2
checked page 3
checked page 4
checked page 5
checked page 6
checked page 7
checked page 8
checked page 9
checked page 10
checked page 11
checked page 12
checked page 13
checked page 14
checked page 15
checked page 16
checked page 17
checked page 18
checked page 19


In [13]:
# Preview the first 5 results in JSON format

# Use json.dumps with argument indent=4 to format data
for i in range(5):
    print(json.dumps(reviews_list[i], indent=4))

{
    "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
    "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
    "source": "The New York Times",
    "headline": {
        "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
        "kicker": null,
        "content_kicker": null,
        "print_headline": "The Attachment Diaries",
        "name": null,
        "seo": null,
        "sub": null
    },
    "keywords": [
        {
            "name": "subject",
            "value": "Movies",
            "rank": 1,
            "major": "N"
        },
        {
            "name": "creative_works",
            "value": "The Attachment Diaries (Movie)",
            "rank": 2,
            "major": "N"
        },
        {
            "name": "persons",
            "value": "Diment, Valentin Javier",
            "rank": 3,
            "major": "N"
        }
    ],
  

In [14]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [15]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

reviews_df["title"] = reviews_df["headline.main"].apply(lambda st: st[st.find('\u2018')+1:st.find("\u2019 Review")])
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",Non

In [16]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string

reviews_df["keywords"] = reviews_df['keywords'].apply(extract_keywords)
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,creative_works: Love & Taxes (Movie);persons: ...,2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,subject: Movies;creative_works: Everybody Love...,2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,"Everybody L

In [17]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_df['title'].to_list()
titles

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

### Access The Movie Database API

In [18]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key
request_url = url + tmdb_key_string

In [19]:
# Create an empty list to store the results
tmdb_movies_list = []

genres = []
spoken_languages = []
production_countries = []
tmdb_movie_dict = {
    'title': [],
    'original_title': [],
    'budget': [],
    'original_language': [],
    'homepage': [],
    'overview': [],
    'popularity': [],
    'runtime': [],
    'revenue': [],
    'release_date': [],
    'vote_average': [],
    'vote_count': [],
    'genres': genres,
    'spoken_languages': spoken_languages,
    'production_countries': production_countries
}



# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for request in titles:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        print(f"Sleeping for 1 second after {request_counter} requests")
        time.sleep(1)  

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    updated_url = url + request + tmdb_key_string
    response = requests.get(updated_url).json()
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    
    try:
        # Get movie id
        movie_id = response["results"][0]["id"]

        # Make a request for a the full movie details
        full_movie_details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?" + tmdb_key_string

        # Execute "GET" request with url
        full_movie_details = requests.get(full_movie_details_url).json()
        
        # Extract the genre names into a list
        genres.append(full_movie_details["genres"][0]["name"])


        # Extract the spoken_languages' English name into a list
        spoken_languages.append(full_movie_details["spoken_languages"][0]["english_name"])

        # Extract the production_countries' name into a list
        production_countries.append(full_movie_details["production_countries"][0]["name"])

        # Add the relevant data to a dictionary and
        tmdb_movie_dict['title'].append(full_movie_details["title"])
        tmdb_movie_dict['original_title'].append(full_movie_details["original_title"])
        tmdb_movie_dict['budget'].append(full_movie_details["budget"])
        tmdb_movie_dict['original_language'].append(full_movie_details["original_language"])
        tmdb_movie_dict['homepage'].append(full_movie_details["homepage"])
        tmdb_movie_dict['overview'].append(full_movie_details["overview"])
        tmdb_movie_dict['popularity'].append(full_movie_details["popularity"])
        tmdb_movie_dict['runtime'].append(full_movie_details["runtime"])
        tmdb_movie_dict['revenue'].append(full_movie_details["revenue"])
        tmdb_movie_dict['release_date'].append(full_movie_details["release_date"])
        tmdb_movie_dict['vote_average'].append(full_movie_details["vote_average"])
        tmdb_movie_dict['vote_count'].append(full_movie_details["vote_count"])
        
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append(tmdb_movie_dict)

        
        # Print out the title that was found
        print(f"Found {request}")
    except:
        print(f"{request} not found")

Found The Attachment Diaries
What’s Love Got to Do With It?’ Probably a Lo not found
Found You Can Live Forever
Found A Tourist’s Guide to Love
Found Other People’s Children
Found One True Loves
Found The Lost Weekend: A Love Story
Found A Thousand and One
Found Your Place or Mine
Found Love in the Time of Fentanyl
Found Pamela, a Love Story
Found In From the Side
Found After Love
Found Alcarràs
Found Nelly & Nadine
Found Lady Chatterley’s Lover
Found The Sound of Christmas
Found The Inspection
Found Bones and All
Found My Policeman
Found About Fate
Found Waiting for Bojangles
Found I Love My Dad
Found A Love Song
Found Alone Together
Found Art of Love
Found The Wheel
Found Thor: Love and Thunder
Found Both Sides of the Blade
Found Fire of Love
Found Love & Gelato
Found Stay Prayed Up
Found Benediction
Found Dinner in America
In a New York Minute not found
Found Anaïs in Love
Found I Love America
Found See You Then
Found La Mami
Found Love After Love
Found Deep Water
Found Lucy and Des

In [20]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
for i in range(5):
    print(json.dumps(tmdb_movies_list[i],indent=4))


{
    "title": [
        "The Attachment Diaries",
        "You Can Live Forever",
        "A Tourist's Guide to Love",
        "Other People's Children",
        "One True Loves",
        "The Lost Weekend: A Love Story",
        "A Thousand and One",
        "Your Place or Mine",
        "Love in the Time of Fentanyl",
        "Pamela, A Love Story",
        "In from the Side",
        "After Love",
        "Alcarr\u00e0s",
        "Nelly and Monsieur Arnaud",
        "Lady Chatterley's Lover",
        "The Sound of Christmas",
        "The Inspection",
        "Bones and All",
        "My Policeman",
        "About Fate",
        "Waiting for Bojangles",
        "I Love My Dad",
        "A Love Song",
        "Alone Together",
        "Art of Love",
        "2099: The Soldier Protocol",
        "Thor: Love and Thunder",
        "Both Sides of the Blade",
        "Fire of Love",
        "Art of Love",
        "Stay Prayed Up",
        "Benediction",
        "Dinner in America",
     

In [22]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df.head()

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,"[The Attachment Diaries, You Can Live Forever,...","[El apego, You Can Live Forever, A Tourist's G...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, ...","[es, en, en, fr, en, en, en, en, en, en, en, e...","[, https://gooddeedentertainment.com/you-can-l...","[Argentina, 1970s. A desperate young woman goe...","[2.765, 35.418, 30.992, 9.043, 15.859, 7.656, ...","[102, 96, 96, 104, 100, 95, 116, 109, 85, 113,...","[0, 15055, 0, 84178, 37820, 77145, 3395595, 0,...","[2021-10-07, 2023-03-24, 2023-04-21, 2022-09-2...","[3.0, 6.6, 6.287, 6.818, 6.5, 6.0, 6.871, 6.28...","[4, 37, 162, 190, 75, 2, 89, 713, 0, 208, 62, ...","[Drama, Drama, Romance, Drama, Romance, Docume...","[Spanish, English, English, French, English, E...","[Argentina, Canada, United States of America, ..."
1,"[The Attachment Diaries, You Can Live Forever,...","[El apego, You Can Live Forever, A Tourist's G...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, ...","[es, en, en, fr, en, en, en, en, en, en, en, e...","[, https://gooddeedentertainment.com/you-can-l...","[Argentina, 1970s. A desperate young woman goe...","[2.765, 35.418, 30.992, 9.043, 15.859, 7.656, ...","[102, 96, 96, 104, 100, 95, 116, 109, 85, 113,...","[0, 15055, 0, 84178, 37820, 77145, 3395595, 0,...","[2021-10-07, 2023-03-24, 2023-04-21, 2022-09-2...","[3.0, 6.6, 6.287, 6.818, 6.5, 6.0, 6.871, 6.28...","[4, 37, 162, 190, 75, 2, 89, 713, 0, 208, 62, ...","[Drama, Drama, Romance, Drama, Romance, Docume...","[Spanish, English, English, French, English, E...","[Argentina, Canada, United States of America, ..."
2,"[The Attachment Diaries, You Can Live Forever,...","[El apego, You Can Live Forever, A Tourist's G...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, ...","[es, en, en, fr, en, en, en, en, en, en, en, e...","[, https://gooddeedentertainment.com/you-can-l...","[Argentina, 1970s. A desperate young woman goe...","[2.765, 35.418, 30.992, 9.043, 15.859, 7.656, ...","[102, 96, 96, 104, 100, 95, 116, 109, 85, 113,...","[0, 15055, 0, 84178, 37820, 77145, 3395595, 0,...","[2021-10-07, 2023-03-24, 2023-04-21, 2022-09-2...","[3.0, 6.6, 6.287, 6.818, 6.5, 6.0, 6.871, 6.28...","[4, 37, 162, 190, 75, 2, 89, 713, 0, 208, 62, ...","[Drama, Drama, Romance, Drama, Romance, Docume...","[Spanish, English, English, French, English, E...","[Argentina, Canada, United States of America, ..."
3,"[The Attachment Diaries, You Can Live Forever,...","[El apego, You Can Live Forever, A Tourist's G...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, ...","[es, en, en, fr, en, en, en, en, en, en, en, e...","[, https://gooddeedentertainment.com/you-can-l...","[Argentina, 1970s. A desperate young woman goe...","[2.765, 35.418, 30.992, 9.043, 15.859, 7.656, ...","[102, 96, 96, 104, 100, 95, 116, 109, 85, 113,...","[0, 15055, 0, 84178, 37820, 77145, 3395595, 0,...","[2021-10-07, 2023-03-24, 2023-04-21, 2022-09-2...","[3.0, 6.6, 6.287, 6.818, 6.5, 6.0, 6.871, 6.28...","[4, 37, 162, 190, 75, 2, 89, 713, 0, 208, 62, ...","[Drama, Drama, Romance, Drama, Romance, Docume...","[Spanish, English, English, French, English, E...","[Argentina, Canada, United States of America, ..."
4,"[The Attachment Diaries, You Can Live Forever,...","[El apego, You Can Live Forever, A Tourist's G...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, ...","[es, en, en, fr, en, en, en, en, en, en, en, e...","[, https://gooddeedentertainment.com/you-can-l...","[Argentina, 1970s. A desperate young woman goe...","[2.765, 35.418, 30.992, 9.043, 15.859, 7.656, ...","[102, 96, 96, 104, 100, 95, 116, 109, 85, 113,...","[0, 15055, 0, 84178, 37820, 77145, 3395595, 0,...","[2021-10-07, 2023-03-24, 2023-04-21, 2022-09-2...","[3.0, 6.6, 6.287, 6.818, 6.5, 6.0, 6.871, 6.28...","[4, 37, 162, 190, 75, 2, 89, 713, 0, 208, 62, ...","[Drama, Drama, Romance, Drama, Romance, Docume...","[Spanish, English, 

### Merge and Clean the Data for Export

In [23]:
# Merge the New York Times reviews and TMDB DataFrames on title
merge_df = pd.merge(reviews_df, tmdb_df, on='title')
merge_df.head()

TypeError: unhashable type: 'list'

In [24]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characers_to_remove = ['[', ']', "'"]

# Loop through the list of columns to fix
for col in columns_to_fix:
    # Convert the column to type 'str'
    merge_df[col] = merge_df[col].astype(str)

    # Loop through characters to remove
    for char in characers_to_remove:
        merge_df[col] = merge_df[col].str.replace(char, '')


# Display the fixed DataFrame
merge_df.head()


NameError: name 'merge_df' is not defined

In [25]:
# Drop "byline.person" column
merge_df.drop(columns=['byline.person'])
merge_df.head()

NameError: name 'merge_df' is not defined

In [26]:
# Delete duplicate rows and reset index
merge_df = merge_df.drop_duplicates().reset_index(drop=True)
merge_df.head()

NameError: name 'merge_df' is not defined

In [27]:
# Export data to CSV without the index
merge_df.to_csv('movie_data.csv', Index=False)

NameError: name 'merge_df' is not defined